In [16]:
import pandas as pd
import itertools
import numpy as np

In [17]:
df = pd.read_csv("step2_out.csv")
df

,layers,branching factor,average error,average error %,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front
0,"radix18,cubic",65536,754.255954,0.002223,10.175163,4445,0.002223,3670024,3145736,nm01f336b04dcbf4beb029,True,False,False
1,"linear_spline,radix8",8192,104949.352036,99.974165,11.458588,199948329,99.974165,8519696,8454160,nm030e3bd77977903d246f,True,False,False
2,"radix22,linear",8192,1983.351534,0.003865,11.843172,7729,0.003865,16973832,16908296,nm070b9d968dde41e02d08,True,False,False
3,"radix18,cubic",524288,498.779692,0.001716,9.504494,3433,0.001716,22020104,17825800,nm0bb6bc7e9506afd62fff,True,False,False
4,"radix22,cubic",8192,2436.144907,0.004566,11.999031,9133,0.004566,17104904,17039368,nm0e181adc07a2acf0c38c,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,"radix8,linear",512,17915.162121,0.021080,15.047385,42160,0.021080,13320,9224,nmf7ab0f6f6bc116cf8e73,True,False,False
236,"linear,linear",256,12856.925219,0.014819,14.565376,29637,0.014819,6160,4112,nmf80200ea7a28f76841dd,True,False,False
237,"linear,linear",32768,948.492536,0.003375,10.705271,6749,0.003375,786448,524304,nmfcc5175d0d33367864b2,True,False,False
238,"radix18,linear",256,12634.277172,0.013458,14.542920,26917,0.013458,1054728,1052680,nmfeb12a132c617dc71dbb,True,False,False


In [18]:
front = df[df.front & (df["average log2 error"] < 15) & (df["size binary search"] < 2000000000)].copy()
front.sort_values("average log2 error")

,layers,branching factor,average error,average error %,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front
185,"radix26,linear",2097152,39.281510,0.001022,5.696685,2043,0.001022,318767112,301989896,nm5224d02427c12afc4e35,True,False,True
165,"radix26,linear",1048576,55.353367,0.001032,6.403406,2064,0.001032,293601288,285212680,nm1051edb8eb3c4850071e,True,False,True
195,"linear_spline,linear",2097152,98.785669,0.002339,6.652626,4679,0.002339,50331664,33554448,nm77680171b4d11b70aa2f,True,False,True
188,"radix22,linear",1048576,135.278753,0.002210,7.340945,4420,0.002210,41943048,33554440,nm62b7c64259882d940086,True,False,True
206,"linear_spline,linear",1048576,142.415857,0.002337,7.348663,4674,0.002337,25165840,16777232,nm9d7314ecfb0606fea357,True,False,True
149,"linear,linear",524288,207.379216,0.002053,8.053351,4105,0.002053,12582928,8388624,nmf1c3a8533d05302435de,True,False,True
223,"linear_spline,linear",262144,302.450670,0.002821,8.749268,5641,0.002821,6291472,4194320,nmd985905d60efa476204b,True,False,True
181,"linear,linear",131072,442.791401,0.003776,9.432260,7552,0.003776,3145744,2097168,nm4ba04214fe8ae1c6a0ef,True,False,True
60,"linear_spline,linear",65536,645.939125,0.003343,10.080427,6686,0.003343,1572880,1048592,nm6d2f6a0dda092b70154a,True,False,True
171,"linear_spline,linear",32768,946.576763,0.003517,10.701635,7034,0.003517,786448,524304,nm238bd4f690b492c7d5e8,True,False,True


In [19]:
front = front.to_dict("record")

In [20]:
retained = []
for rmi1 in front:
    for rmi2 in front:
        if rmi1["layers"] == rmi2["layers"] and rmi1["branching factor"] == rmi2["branching factor"]:
            continue
        
        if abs(rmi1["average log2 error"] - rmi2["average log2 error"]) < 0.001:
            if rmi1["size binary search"] > rmi2["size binary search"]:
                break
                
        if abs(rmi1["size binary search"] - rmi2["size binary search"]) < 1000:
            if rmi1["average log2 error"] > rmi2["average log2 error"]:
                break
    else:
        retained.append(rmi1)
        
retained = pd.DataFrame(retained).sort_values("average log2 error")
retained

,layers,branching factor,average error,average error %,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front
8,"radix26,linear",2097152,39.281510,0.001022,5.696685,2043,0.001022,318767112,301989896,nm5224d02427c12afc4e35,True,False,True
4,"radix26,linear",1048576,55.353367,0.001032,6.403406,2064,0.001032,293601288,285212680,nm1051edb8eb3c4850071e,True,False,True
11,"linear_spline,linear",2097152,98.785669,0.002339,6.652626,4679,0.002339,50331664,33554448,nm77680171b4d11b70aa2f,True,False,True
9,"radix22,linear",1048576,135.278753,0.002210,7.340945,4420,0.002210,41943048,33554440,nm62b7c64259882d940086,True,False,True
12,"linear_spline,linear",1048576,142.415857,0.002337,7.348663,4674,0.002337,25165840,16777232,nm9d7314ecfb0606fea357,True,False,True
3,"linear,linear",524288,207.379216,0.002053,8.053351,4105,0.002053,12582928,8388624,nmf1c3a8533d05302435de,True,False,True
16,"linear_spline,linear",262144,302.450670,0.002821,8.749268,5641,0.002821,6291472,4194320,nmd985905d60efa476204b,True,False,True
7,"linear,linear",131072,442.791401,0.003776,9.432260,7552,0.003776,3145744,2097168,nm4ba04214fe8ae1c6a0ef,True,False,True
1,"linear_spline,linear",65536,645.939125,0.003343,10.080427,6686,0.003343,1572880,1048592,nm6d2f6a0dda092b70154a,True,False,True
5,"linear_spline,linear",32768,946.576763,0.003517,10.701635,7034,0.003517,786448,524304,nm238bd4f690b492c7d5e8,True,False,True


In [21]:
retained["diff"] = [1000] + list(np.diff(retained["average log2 error"].to_numpy()))
selected = retained.sort_values("diff", ascending=False)[0:10]
selected = selected.sort_values("size binary search", ascending=False)
selected

,layers,branching factor,average error,average error %,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front,diff
8,"radix26,linear",2097152,39.281510,0.001022,5.696685,2043,0.001022,318767112,301989896,nm5224d02427c12afc4e35,True,False,True,1000.000000
4,"radix26,linear",1048576,55.353367,0.001032,6.403406,2064,0.001032,293601288,285212680,nm1051edb8eb3c4850071e,True,False,True,0.706721
9,"radix22,linear",1048576,135.278753,0.002210,7.340945,4420,0.002210,41943048,33554440,nm62b7c64259882d940086,True,False,True,0.688319
3,"linear,linear",524288,207.379216,0.002053,8.053351,4105,0.002053,12582928,8388624,nmf1c3a8533d05302435de,True,False,True,0.704687
16,"linear_spline,linear",262144,302.450670,0.002821,8.749268,5641,0.002821,6291472,4194320,nmd985905d60efa476204b,True,False,True,0.695917
7,"linear,linear",131072,442.791401,0.003776,9.432260,7552,0.003776,3145744,2097168,nm4ba04214fe8ae1c6a0ef,True,False,True,0.682992
1,"linear_spline,linear",65536,645.939125,0.003343,10.080427,6686,0.003343,1572880,1048592,nm6d2f6a0dda092b70154a,True,False,True,0.648166
5,"linear_spline,linear",32768,946.576763,0.003517,10.701635,7034,0.003517,786448,524304,nm238bd4f690b492c7d5e8,True,False,True,0.621208
14,"linear_spline,linear",16384,1386.292729,0.005959,11.295717,11919,0.005959,393232,262160,nmc64eb7f62650bbe7dfff,True,False,True,0.594083
2,"linear_spline,linear",8192,2038.996383,0.004575,11.875593,9150,0.004575,196624,131088,nmaac4a789ceba69493dce,True,False,True,0.579875


In [22]:
DATASET = "fb_200M_uint64"
tmp = selected.reset_index().drop(columns="index").reset_index()
tmp["namespace"] = tmp.apply(lambda x: DATASET + "_" + str(x["index"]), axis=1)
spec = {"configs" : tmp[["layers", "branching factor", "namespace", "binary"]].to_dict("record")}

import json
print(json.dumps(spec))

{"configs": [{"layers": "radix26,linear", "branching factor": 2097152, "namespace": "fb_200M_uint64_0", "binary": true}, {"layers": "radix26,linear", "branching factor": 1048576, "namespace": "fb_200M_uint64_1", "binary": true}, {"layers": "radix22,linear", "branching factor": 1048576, "namespace": "fb_200M_uint64_2", "binary": true}, {"layers": "linear,linear", "branching factor": 524288, "namespace": "fb_200M_uint64_3", "binary": true}, {"layers": "linear_spline,linear", "branching factor": 262144, "namespace": "fb_200M_uint64_4", "binary": true}, {"layers": "linear,linear", "branching factor": 131072, "namespace": "fb_200M_uint64_5", "binary": true}, {"layers": "linear_spline,linear", "branching factor": 65536, "namespace": "fb_200M_uint64_6", "binary": true}, {"layers": "linear_spline,linear", "branching factor": 32768, "namespace": "fb_200M_uint64_7", "binary": true}, {"layers": "linear_spline,linear", "branching factor": 16384, "namespace": "fb_200M_uint64_8", "binary": true}, {"